In [2]:
import json
# from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
# import sqlite3
import http.client
import time
import collections
from collections import defaultdict

In [3]:
data_folder = '../../components/Tools/NASHBoard/Data/'

In [4]:
f = open('../api_keys.json')
data = json.load(f)
API_key = data["SportRadar"]

# establish connection to API
conn = ''
def get_conn():
    global conn 
    conn = http.client.HTTPSConnection("api.sportradar.us")
get_conn()

# function to get and minimally format data given url
def get_data(url):
    conn.request("GET", url)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

In [ ]:
# GET INJURY DATA
injury_data = get_data("/nba/trial/v7/en/league/injuries.json?api_key=%s" %API_key)
# reformat injuries data
injuries_list = []

for i, row in enumerate(injury_data['teams']):
    team = row['market'] + ' ' + row['name']
    for player in row['players']:
        injured = {}
        injured['id'] = i
        injured['team_name'] = team
        injured.update({k: player[k] for k in ('full_name', 'primary_position','reference')})
        injured.update({k: player['injuries'][0][k] for k in ('comment', 'status','start_date','update_date')})
        injured['player_id'] = injured.pop('reference')
        injuries_list.append(injured)

# store in json
with open(data_folder + 'injuries.json', 'w') as outfile:
    json.dump(injuries_list, outfile)

In [5]:
# UPDATE WEEKLY GAMES
season = "2022"
get_conn()
curr_season_schedule = get_data("/nba/trial/v7/en/games/" + season + "/REG/schedule.json?api_key=%s" %API_key)
games_list = []
future_games_list = []
num_games = len(curr_season_schedule['games'])
for i, row in enumerate(curr_season_schedule['games']):
    if i % 100 == 0:
        print('game',i,'of',num_games)
        
    game = {}
    if row['status'] == 'postponed' or row['status'] == 'cancelled':
        continue
    else:
        game['time_zone'] = row['time_zones']['venue']
        game['network'] = row['broadcasts'][0]['network'] if row.get('broadcasts') else 'missing'
        try:
            game['locale'] = row['broadcasts'][0]['locale']
        except:
            game['locale'] = 'home'
        game['home'] = row['home']['name']
        game['away'] = row['away']['name']
        # add to future games list if not yet played
        if row['status'] == 'scheduled':
            game.update({k: row[k] for k in ('id', 'scheduled')})
            future_games_list.append(game)
        # else, add to played games list
        else:
            # try:
            game.update({k: row[k] for k in ('id', 'scheduled','home_points','away_points')})
            games_list.append(game)
    
# add to current year schedule jsons
with open(data_folder + 'season_' + season + '_schedule.json', 'w') as outfile:
    json.dump(games_list, outfile)
with open(data_folder + 'season_' + season + '_remaining_schedule.json', 'w') as outfile:
    json.dump(future_games_list, outfile)

game 0 of 1230
game 50 of 1230
game 100 of 1230
game 150 of 1230
game 200 of 1230
game 250 of 1230
game 300 of 1230
game 350 of 1230
game 400 of 1230
game 450 of 1230
game 500 of 1230
game 550 of 1230
game 600 of 1230
game 650 of 1230
game 700 of 1230
game 750 of 1230
game 800 of 1230
game 850 of 1230
game 900 of 1230
game 950 of 1230
game 1000 of 1230
game 1050 of 1230
game 1100 of 1230
game 1150 of 1230
game 1200 of 1230


In [6]:
# GET UPDATED TEAM SPLITS
with open(data_folder + 'teams.json', "r") as read_content:
    team_json = json.load(read_content)
team_ids = {t['teamName']:t['sr-id'] for t in team_json}

year = '2022'
get_conn()
team_splits = {}
split_splits = [0,4,10]

for t,i in team_ids.items():
    print(t)
    time.sleep(1)
    data = get_data("/nba/trial/v7/en/seasons/" + year + "/REG/teams/" + i + "/splits/ingame.json?api_key=%s"%API_key)
    total_games = 0
    weighted_opp_splits_list = []
    weighted_self_splits_list = []
    for s in split_splits:
        split_games = data['splits'][s]['opponents']['total']['games_played']
        total_games = total_games + split_games
        opp_splits = data['splits'][s]['opponents']['average']
        weighted_opp_splits = {s:opp_splits[s] * split_games for s in list(opp_splits.keys())}
        weighted_opp_splits_list.append(weighted_opp_splits)

        self_splits = data['splits'][s]['own_record']['average']
        weighted_self_splits = {s:self_splits[s] * split_games for s in list(self_splits.keys())}
        weighted_self_splits_list.append(weighted_self_splits)

    list_keys = [k for k in list(weighted_opp_splits.keys())]
    total_weighted_opp_splits = dict(zip(list_keys,[0] * len(list_keys)))
    for w in weighted_opp_splits_list:
        for k in list_keys:
            total_weighted_opp_splits[k] += w[k]
    total_weighted_opp_splits = {k:round(i/total_games,2) for k,i in total_weighted_opp_splits.items()}

    total_weighted_self_splits = dict(zip(list_keys,[0] * len(list_keys)))
    for w in weighted_self_splits_list:
        for k in list_keys:
            total_weighted_self_splits[k] += w[k]
    total_weighted_self_splits = {k:round(i/total_games,2) for k,i in total_weighted_self_splits.items()}

    all_splits_dict = {'opponent':total_weighted_opp_splits,'self':total_weighted_self_splits}
    team_splits[t] = all_splits_dict
# team_splits

NameError: name 'team_ids' is not defined

In [ ]:
with open(data_folder + 'team_splits.json', 'w') as outfile:
    json.dump(team_splits, outfile)

In [ ]:
# UPDATE PLAYER BOX SCORES?